In [7]:
## Installing Libraries
!pip install langchain langchain-core langchain-community langchain-huggingface requests -q


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
## Imorting necessary libraries
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.agents import initialize_agent, Tool, tool
from langchain.agents.agent_types import AgentType
from langchain_core.messages import HumanMessage
import requests
import json
import functools

In [84]:
## Creating a python tool 

@tool
def multiply(a: int, b: int) -> int:
    '''Given two numbers a and b this tool returns their product'''
    return a * b

In [64]:
## Tool testing
print(multiply.invoke({'a' : 3, 'b' : 4}))
print(multiply.name)
print(multiply.description)
print(multiply.args)

12
multiply
Given two numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [66]:
## Initializing HuggingFace LLM
llm = HuggingFaceEndpoint(
    repo_id= 'meta-llama/Llama-3.3-70B-Instruct'
)

## Initializing model
model = ChatHuggingFace(llm = llm)

In [67]:
## tool binding with LLM
llm_with_tools = model.bind_tools([multiply])

#### Method 1:  Conventional method - without tools calling

In [ ]:
## Tool calling
llm_with_tools.invoke("What is 8 multiplied by 7 ?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": "8", "b": "7"}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-6a98c6c46a8447d2a1355b557d8e671a', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 204, 'total_tokens': 232}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--750c4453-6009-4b69-b7f9-f981389fec72-0', tool_calls=[{'name': 'multiply', 'args': {'a': '8', 'b': '7'}, 'id': 'chatcmpl-tool-6a98c6c46a8447d2a1355b557d8e671a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 204, 'output_tokens': 28, 'total_tokens': 232})

In [69]:
## Cleaning the call
json_argument = json.loads(llm_with_tools.invoke("What is 8 multiplied by 7 ?").additional_kwargs['tool_calls'][0]['function']['arguments'])
print(json_argument)

{'a': '8', 'b': '7'}


In [70]:
## Doing calculation
multiply.invoke(json_argument)

56

#### Method 2:  Efficient method - with tools calling

In [99]:
## Tool calling
response = llm_with_tools.invoke("Can you multiply 3 with 10?")
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": "3", "b": "10"}', 'name': 'multiply', 'description': None}, 'id': 'chatcmpl-tool-ec7dadf5971c40ae9f66d2fc9253726c', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 204, 'total_tokens': 232}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': '', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--6a57285c-cc11-4b74-92ef-192e73c04b5e-0', tool_calls=[{'name': 'multiply', 'args': {'a': '3', 'b': '10'}, 'id': 'chatcmpl-tool-ec7dadf5971c40ae9f66d2fc9253726c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 204, 'output_tokens': 28, 'total_tokens': 232})

In [94]:
## Since the reposnse structure we are getting is incompatable to what langchain's invoke() method accepts, we are reconstructing a function by defining a new function:
def extract_and_normalize_tool_call(response):
    """Extract the first tool call from LLM output and normalize it for LangChain invoke"""
    tool_call_raw = response.additional_kwargs.get("tool_calls", [])[0]

    return {
        "name": tool_call_raw["function"]["name"],
        "args": json.loads(tool_call_raw["function"]["arguments"]),
        "description": tool_call_raw["function"].get("description"),
        "id": tool_call_raw["id"],
        "type": "tool_call"
    }

In [100]:
## Getting the required structured output from LLM
structured_tool_call = extract_and_normalize_tool_call(response=response)
structured_tool_call

{'name': 'multiply',
 'args': {'a': '3', 'b': '10'},
 'description': None,
 'id': 'chatcmpl-tool-ec7dadf5971c40ae9f66d2fc9253726c',
 'type': 'tool_call'}

In [98]:
multiply.invoke(structured_tool_call)

ToolMessage(content='30', name='multiply', tool_call_id='chatcmpl-tool-c2ad996a53544521834d0eb736e5ca9c')